In [11]:
# %% Import libraries
import pandas as pd
import numpy as np

# %% Load data (replace path or use an existing DataFrame `df`)
# If you already have a DataFrame named `NYC_Insp_Results`, skip this read_csv line.
df = pd.read_csv("/Users/shanmukh/Downloads/DOHMH_New_York_City_Restaurant_Inspection_Results.csv", parse_dates=["INSPECTION DATE"])

# If the date column didn't parse above, force-parse it:
if not np.issubdtype(df["INSPECTION DATE"].dtype, np.datetime64):
    df["INSPECTION DATE"] = pd.to_datetime(df["INSPECTION DATE"], errors="coerce")

# %% Clean / prepare columns (optional but recommended)
# Ensure SCORE is numeric (SQL would treat it numeric for comparisons)
df["SCORE"] = pd.to_numeric(df["SCORE"], errors="coerce")

# Trim whitespace in string columns to avoid mismatches
for col in ["INSPECTION TYPE", "GRADE"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# %% Apply the same WHERE logic as your SQL
# Recreate the three inspection-type groups used in the SQL and the grade list
reinspection_types = [
    "Cycle Inspection / Re-inspection",
    "Pre-permit (Operational) / Re-inspection"
]

initial_types = [
    "Cycle Inspection / Initial Inspection",
    "Pre-permit (Operational) / Initial Inspection"
]

reopening_types = [
    "Pre-permit (Operational) / Reopening Inspection",
    "Cycle Inspection / Reopening Inspection"
]

grade_allowed = ["A", "B", "C", "P", "Z"]

# Build boolean masks
mask_reinspection = df["INSPECTION TYPE"].isin(reinspection_types)
mask_initial_and_low_score = df["INSPECTION TYPE"].isin(initial_types) & (df["SCORE"] <= 13)
mask_reopening = df["INSPECTION TYPE"].isin(reopening_types)
mask_grade = df["GRADE"].isin(grade_allowed)

# Combined filter: (reinspection OR (initial AND score<=13) OR reopening) AND grade in (...)
combined_mask = (mask_reinspection | mask_initial_and_low_score | mask_reopening) & mask_grade

filtered = df.loc[combined_mask].copy()

# %% Group by CAMIS and get the most recent inspection date (equivalent to MAX([INSPECTION DATE]))
RecentInspDate = (
    filtered
    .dropna(subset=["CAMIS", "INSPECTION DATE"])        # ensure CAMIS and date are present
    .groupby("CAMIS", as_index=False)["INSPECTION DATE"]
    .max()
    .rename(columns={"INSPECTION DATE": "MostRecentInspDate"})
)

# %% Inspect result
print("Rows in RecentInspDate:", RecentInspDate.shape[0])
RecentInspDate.head(10)

# %% (Optional) Merge back to original results if you want the rows with the most recent date per CAMIS
# This returns the rows from filtered that correspond to the MostRecentInspDate by CAMIS.
latest_rows = filtered.merge(
    RecentInspDate,
    left_on=["CAMIS", "INSPECTION DATE"],
    right_on=["CAMIS", "MostRecentInspDate"],
    how="inner",
    suffixes=("", "_mostrecent")
)

# %% Save if needed
RecentInspDate.to_csv("RecentInspDate.csv", index=False)
print("Saved RecentInspDate.csv")


Rows in RecentInspDate: 25023
Saved RecentInspDate.csv


In [13]:
# %% Assume you already have:
# - df = NYC_Insp_Results DataFrame (loaded & cleaned)
# - RecentInspDate DataFrame (from the earlier step)

# Reuse the inspection type groups
reinspection_types = [
    "Cycle Inspection / Re-inspection",
    "Pre-permit (Operational) / Re-inspection"
]

initial_types = [
    "Cycle Inspection / Initial Inspection",
    "Pre-permit (Operational) / Initial Inspection"
]

reopening_types = [
    "Pre-permit (Operational) / Reopening Inspection",
    "Cycle Inspection / Reopening Inspection"
]

# Build masks
mask_reinspection = df["INSPECTION TYPE"].isin(reinspection_types)
mask_initial_and_low_score = df["INSPECTION TYPE"].isin(initial_types) & (df["SCORE"] <= 13)
mask_reopening = df["INSPECTION TYPE"].isin(reopening_types)

# Combined filter (same as SQL WHERE)
combined_mask = mask_reinspection | mask_initial_and_low_score | mask_reopening

# %% Join RecentInspDate with inspection results (SQL: JOIN)
merged = df.merge(
    RecentInspDate,
    left_on=["CAMIS", "INSPECTION DATE"],
    right_on=["CAMIS", "MostRecentInspDate"],
    how="inner",
    suffixes=("", "_recent")
)

# %% Apply filter from WHERE clause
final_df = merged.loc[combined_mask, ["CAMIS", "DBA", "MostRecentInspDate", "GRADE", "INSPECTION TYPE", "SCORE"]].drop_duplicates()

# %% Rename columns for clarity (like SQL alias)
final_df = final_df.rename(columns={"DBA": "Name"})

# %% Inspect result
print("Final dataset shape:", final_df.shape)
final_df.head(10)

# %% Save if needed
final_df.to_csv("Final_Inspection_Data.csv", index=False)
print("Saved Final_Inspection_Data.csv")


Final dataset shape: (20039, 6)
Saved Final_Inspection_Data.csv
